# Automatic Model Tuning : Warm Starting Tuning Jobs
_**Using warm start to tune a marketing predictions algorithm.**_


## Contents

1. [Introduction](#intro)
2. [Setup](#setup)
3. [Data preparation](#data-prep)
4. [Launch hyperparameter tuning job](#hyperparam1)
5. [Evaluate results of hyperparameter tuning job](#eval1)
6. [Data cleaning](#data-cleaning)
7. [Launch hyperparameter tuning job using warm start](#hyperparam2)
8. [Evaluate results of warm start hyperparameter tuning job](#eval2)
9. [Get the best model](#best-model)
10. [Conclusion](#conclusion)


---

## Introduction <a class="anchor" id="intro"></a>

Selecting the right hyperparameter values for your machine learning model can be difficult. The right answer is dependent on your data; some algorithms have many different hyperparameters that can be tweaked; some are very sensitive to the hyperparameter values selected; and most have a non-linear relationship between model fit and hyperparameter values. 

Amazon SageMaker Automatic Model Tuning helps with automating the hyperparameter tuning process. In many occasions the tuning process is iterative and requires to run multiple tuning jobs after analyzing the results to get the best objective metric.

Warm start configuration allows you to create a new tuning job with the learning gathered in a parent tuning job by specifying up to 5 parent tuning jobs. If a warm start configuration is specified, Automatic Model Tuning will load the previous hyperparameter set and objective metrics values to warm start the new tuning job. This means, you can continue optimizing your model from the point you finished your previous tuning job experiment.

This notebook will train a model which can be used to predict if a customer will enroll for a term deposit at a bank, after one or more phone calls, based on data from the [Bank Marketing data set](https://archive.ics.uci.edu/ml/datasets/bank+marketing). Hyperparameter tuning, leveraging the warm start feature, will be used in order to try multiple hyperparameter settings and produce the best model.

---

## Setup <a class="anchor" id="setup"></a>

First, set up some of the basic resources required to run SageMaker. This includes:

- The S3 bucket and prefix that you want to use for your training and model data. This should be within the same region as your SageMaker notebook instance.
- The IAM role used to give training access to your data.

In [ ]:
import sagemaker
import boto3

# Get the role associated with this SageMaker notebook.
role = sagemaker.get_execution_role()
print("Role name: {}".format(role))

# Start a session and create a new S3 bucket for storing the training data.
sess = sagemaker.Session()
bucket=sess.default_bucket()
print("Bucket name: {}".format(bucket))

# Set a prefix for storing your data.
prefix = 'DEMO-marketing-warmstart'

Navigate to your S3 buckets using the AWS Management Console. You should see a new, empty bucket created by the above script with the same bucket name as printed in the results. The bucket name should be similar to *sagemaker-us-east-1-002268754244*. You may need to wait several minutes for the bucket creation to complete.

<img src="img/workbook_screenshot1.PNG" alt="A bucket is automatically created!" width="600"/>

## Data preparation <a class="anchor" id="data-prep"></a>
Start by downloading the [Bank Marketing data set](https://archive.ics.uci.edu/ml/datasets/bank+marketing) from UCI's ML Repository. Then read in the data and display it as a table.

In [ ]:
# Download and unzip the marketing data set.
!wget -N https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
!unzip -o bank-additional.zip

In [ ]:
import pandas as pd
import numpy as np

# Read and display the marketing data set.
data = pd.read_csv('./bank-additional/bank-additional-full.csv', sep=';')
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 50)         # Keep the output on one page
data

Let's take a look at the data.  At a high level, we can see:

* We have a little over 40K customer records, and 20 features for each customer.
* The features are mixed; some numeric, some categorical.
* The data appears to be sorted, at least by `time` and `contact`.

_**Specifics on each of the features:**_

*Demographics:*
* `age`: Customer's age (numeric)
* `job`: Type of job (categorical: 'admin.', 'services', ...)
* `marital`: Marital status (categorical: 'married', 'single', ...)
* `education`: Level of education (categorical: 'basic.4y', 'high.school', ...)

*Past customer events:*
* `default`: Has credit in default? (categorical: 'no', 'unknown', ...)
* `housing`: Has housing loan? (categorical: 'no', 'yes', ...)
* `loan`: Has personal loan? (categorical: 'no', 'yes', ...)

*Past direct marketing contacts:*
* `contact`: Contact communication type (categorical: 'cellular', 'telephone', ...)
* `month`: Last contact month of year (categorical: 'may', 'nov', ...)
* `day_of_week`: Last contact day of the week (categorical: 'mon', 'fri', ...)
* `duration`: Last contact duration, in seconds (numeric). Important note: If duration = 0 then `y` = 'no'.
 
*Campaign information:*
* `campaign`: Number of contacts performed during this campaign and for this client (numeric, includes last contact)
* `pdays`: Number of days that passed by after the client was last contacted from a previous campaign (numeric)
* `previous`: Number of contacts performed before this campaign and for this client (numeric)
* `poutcome`: Outcome of the previous marketing campaign (categorical: 'nonexistent','success', ...)

*External environment factors:*
* `emp.var.rate`: Employment variation rate - quarterly indicator (numeric)
* `cons.price.idx`: Consumer price index - monthly indicator (numeric)
* `cons.conf.idx`: Consumer confidence index - monthly indicator (numeric)
* `euribor3m`: Euribor 3 month rate - daily indicator (numeric)
* `nr.employed`: Number of employees - quarterly indicator (numeric)

*Target variable:*
* `y`: Has the client subscribed a term deposit? (binary: 'yes','no')

Normally you would clean the data before applying a machine learning algorithm, but we want to get the first hyperparameter tuning job up and running as quickly as possible, so let's only do the bare minimum. This means only taking some necessary formatting steps to ensure that the algroithm, XGBoost, will accept the data as input.

One of the data formats accepted by XGBoost is CSV (comma-separated value). The CSV should be structured such that the rows are observations, the first column is the target variable, and the remaining columns are the features. In addition, the CSV file should contain only numeric data and no header. For more details, [read the documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html#InputOutput-XGBoost) on the input and output format of XGBoost. If the data is not formatted correctly, you can expect to see an error similar to the image below.

<img src="img/workbook_screenshot2.PNG" alt="Error message for incorrect data input." width="600"/>

First, let's ensure that the data only contains numeric values. This means converting all categorical variables (e.g. `marital` column) into sets of indicators. A new column is created for each value (e.g. marital_divorced, marital_married, marital_single, and marital_unknown), and for each user (i.e. row) a 1 marks the relevant value while the rest of the columns contain 0's. This method is known as one-hot encoding.

In [ ]:
# Convert categorical variables to sets of indicators
model_data = pd.get_dummies(data)
model_data

Since our target variable was categorical (i.e. 'yes' or 'no'), the one-hot encoding method resulted in two columns: `y_yes` and `y_no`, which are exact opposites of each other. XGBoost only accepts one target variable column and expects that to be the first column in the data set, so the data set needs to be transformed to reflect these requirements.

Afterwards the data can be shuffled and split into a training set (70%) and a validation set (30%). We won't need a test set for this exercise. Finally, the data can be saved as CSV files without headers or index columns.

In [ ]:
# Remove excess target variable column and copy the remaining target variable column to be the first column
model_data = pd.concat([model_data['y_yes'], model_data.drop(['y_no', 'y_yes'], axis=1)], axis=1)

# Split data into 70% training set and 30% validation set
train_data = model_data.sample(frac=0.7, random_state=1729)
validation_data = model_data.drop(train_data.index)

# # Save to CSV format. Drop headers and index.
train_data.to_csv('train.csv', index=False, header=False)
validation_data.to_csv('validation.csv', index=False, header=False)

To build a SageMaker model, the training and validation data need to be stored in an S3 bucket. Remember that the bucket should be in the same AWS region as your SageMaker instance.

After uploading the data to S3, we need to tell SageMaker where to find the data and where to save the output.

In [ ]:
# Set the file locations
s3train = 's3://{}/{}/train/'.format(bucket, prefix)
s3validation = 's3://{}/{}/validation/'.format(bucket, prefix)

# Upload the data to S3
!aws s3 cp train.csv $s3train
!aws s3 cp validation.csv $s3validation

In [ ]:
# Tell SageMaker where to find the input data and where to store the output
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)
s3_input_train = sagemaker.session.s3_input(s3train, distribution='FullyReplicated', 
                        content_type='text/csv', s3_data_type='S3Prefix')
s3_input_validation = sagemaker.session.s3_input(s3validation, distribution='FullyReplicated', 
                             content_type='text/csv', s3_data_type='S3Prefix')

## Launch hyperparameter tuning job <a class="anchor" id="hyperparam1"></a>
Then we configure the training jobs the hyperparameter tuning job will launch by defining a JSON object that specifies following information:
* The container image for the algorithm (XGBoost)
* The input configuration for the training and validation data
* Configuration for the output of the algorithm
* The values of any algorithm hyperparameters that are not tuned in the tuning job (StaticHyperparameters)
* The type and number of instances to use for the training jobs
* The stopping condition for the training jobs

Again, since we are using built-in XGBoost algorithm here, it emits two predefined metrics: *validation:auc* and *train:auc*, and we elected to monitor *validation_auc* as you can see above. One thing to note is if you bring your own algorithm, your algorithm emits metrics by itself. In that case, you'll need to add a MetricDefinition object here to define the format of those metrics through regex, so that SageMaker knows how to extract those metrics.

In [ ]:
# Fetch the XGBoost container for your region
my_region = boto3.session.Session().region_name
container = sagemaker.amazon.amazon_estimator.get_image_uri(my_region, 'xgboost')

# Set up an XGBoost trainer
xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path=s3_output_location,
                                    sagemaker_session=sess)

# Set initial values for all hyperparameters
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

XGBoost algorithm has dozens of hyperparameters and we need to pick the right values for those hyperparameters in order to achieve the desired model training results. Since which hyperparameter setting can lead to the best result depends on the dataset as well, it is almost impossible to pick the best hyperparameter setting without searching for it, and a good search algorithm can search for the best hyperparameter setting in an automated and effective way.

We will use SageMaker hyperparameter tuning to automate the searching process effectively. Specifically, we specify a range, or a list of possible values in the case of categorical hyperparameters, for each of the hyperparameter that we plan to tune. SageMaker hyperparameter tuning will automatically launch multiple training jobs with different hyperparameter settings, evaluate results of those training jobs based on a predefined "objective metric", and select the hyperparameter settings for future attempts based on previous results. For each hyperparameter tuning job, we will give it a budget (max number of training jobs) and it will complete once that many training jobs have been executed.

Now we configure the hyperparameter tuning job by defining a JSON object that specifies following information:
* The ranges of hyperparameters we want to tune
* Number of training jobs to run in total and how many training jobs should be run simultaneously. More parallel jobs will finish tuning sooner, but may sacrifice accuracy. We recommend you set the parallel jobs value to less than 10% of the total number of training jobs (we'll set it higher just for this example to keep it short).
* The objective metric that will be used to evaluate training results, in this example, we select *validation:auc* to be the objective metric and the goal is to maximize the value throughout the hyperparameter tuning process. One thing to note is the objective metric has to be among the metrics that are emitted by the algorithm during training. In this example, the built-in XGBoost algorithm emits a bunch of metrics and *validation:auc* is one of them. If you bring your own algorithm to SageMaker, then you need to make sure whatever objective metric you select, your algorithm actually emits it.

We will tune four hyperparameters in this examples:
* *eta*: Step size shrinkage used in updates to prevent overfitting. After each boosting step, you can directly get the weights of new features. The eta parameter actually shrinks the feature weights to make the boosting process more conservative. 
* *alpha*: L1 regularization term on weights. Increasing this value makes models more conservative. 
* *min_child_weight*: Minimum sum of instance weight (hessian) needed in a child. If the tree partition step results in a leaf node with the sum of instance weight less than min_child_weight, the building process gives up further partitioning. In linear regression models, this simply corresponds to a minimum number of instances needed in each node. The larger the algorithm, the more conservative it is. 
* *max_depth*: Maximum depth of a tree. Increasing this value makes the model more complex and likely to be overfitted. 

In the following cell you are required to enter some code following the instructions.

In [ ]:
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner

# Set the search range for each hyperparameter
# TODO set the 'hyperparameter_ranges' variable
# this is a dictionary {} with 4 members defining the ranges of the hyperparameters to search:
# 'eta' must be set as a ContinuousParameter() ranging from 0 to 1
# 'alpha' must be set as a ContinuousParameter() ranging from 0 to 2
# 'min_child_weight' must be set as a ContinousParameter() ranging from 1 to 10
# 'max_depth' must be set as an IntegerParameter() raging from 1 to 10
hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                         'alpha': ContinuousParameter(0, 2),
                         'min_child_weight': ContinuousParameter(1, 10),
                         'max_depth': IntegerParameter(1, 10)}

# Choose the objective metric to maximize and set up the tuner
objective_metric_name = 'validation:auc'
# TODO create the HyperparameterTuner with the following arguments:
# xgb - the estimator we created previously,
# the objective_metric_name
# an objective_type = 'Maximize'
# 5 max_jobs
# 3 max_parallel_jobs
tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            objective_type='Maximize',
                            max_jobs=5,
                            max_parallel_jobs=3)

Now we can launch a hyperparameter tuning job by calling fit in tuner. After the hyperparameter tuning job is created, we can go to SageMaker console (under Training -> Training jobs) to track the progress of the hyperparameter tuning job until it is completed.

In [ ]:
tuner.fit({'train': s3_input_train, 'validation': s3_input_validation},include_cls_metadata=False)

## Evaluate results of hyperparameter tuning job <a class="anchor" id="eval1"></a>

_You will be unable to successfully run this section until the tuning job completes. Check the progress of the tuning job in the console, or run the first cell below to receive the status. While waiting, feel free to start [cleaning the data](#data-cleaning) in preparation for the second tuning job._

In [ ]:
# Run this cell to check current status of hyperparameter tuning job
sage_client = boto3.Session().client('sagemaker')
job_name = tuner._current_job_name
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
print("Current status: {}".format(status))
if status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
print("{} training jobs have completed".format(job_count))

Once the tuning job finishes, we can bring in a table of metrics.

In [ ]:
tuning_job_name = tuner._current_job_name

tuner_parent_metrics = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
if not tuner_parent_metrics.dataframe().empty:
    df_parent = tuner_parent_metrics.dataframe().sort_values(['FinalObjectiveValue'], ascending=False)
    
df_parent

You can analyze the results deeper by using [HPO_Analyze_TuningJob_Results.ipynb notebook](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/hyperparameter_tuning/analyze_results). Here, we will just plot how the objective metric changes overtime as the tuning progresses.

In [ ]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

import pandas as pd

df_parent_objective_value = df_parent[df_parent['FinalObjectiveValue'] > -float('inf')]

p = figure(plot_width=900, plot_height=400, x_axis_type='datetime',x_axis_label='datetime', y_axis_label=objective_metric_name)
p.circle(source=df_parent_objective_value, x='TrainingStartTime', y='FinalObjectiveValue', color='black')

show(p)

## Data cleaning <a class="anchor" id="data-cleaning"></a>

Depending on how your first hyperparameter tuning job went, you may or may not want to try another tuning job to see whether the model quality can be further improved. When you decide to run another tuning job, you would want to leverage what has been known about the search space from the completed tuning job. In that case, you can create a new hyperparameter tuning job, while warm starting it using the completed tuning job, instead of starting from scratch.

It is optional to change the data before starting the second tuning job. Since we only applied a couple of reformatting steps before running the first tuning job - skipping over the standard data cleaning activities - go ahead and perform some data cleaning while waiting for the results of the first tuning job. 

To get started, one question to ask yourself is whether certain features will add value in your final use case. For example, if your goal is to deliver the best prediction, then will you have access to that data at the moment of prediction? Knowing it's raining is highly predictive for umbrella sales, but forecasting weather far enough out to plan inventory on umbrellas is probably just as difficult as forecasting umbrella sales without knowledge of the weather. So, including this in your model may give you a false sense of precision. Following this logic, let's remove the economic features and duration from our data as they would need to be forecasted with high precision to use as inputs in future predictions.

The rest of the cell has been left blank for you to apply your own data cleaning steps. Below are some ideas if you are not sure what to do:
1. Many records have the value of "999" for pdays, number of days that passed by after a client was last contacted. It is very likely to be a magic number to represent that no contact was made before. Considering that, we create a new column called "no_previous_contact", then grant it value of "1" when pdays is 999 and "0" otherwise.
1. In the "job" column, there are categories that mean the customer is not working, e.g., "student", "retire", and "unemployed". Since it is very likely whether or not a customer is working will affect his/her decision to enroll in the term deposit, we generate a new column to show whether the customer is working based on "job" column.

In [ ]:
# Drop columns requiring accurate future forecasts
clean_data = data.drop(['duration', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'], axis=1)

# Apply your own data cleaning steps below on the clean_data variable


In [ ]:
# DATA CLEANING (CHEAT SHEET FOR INSTRUCTORS - SHOULD NOT BE INCLUDED IN FINAL SHEET FOR PARTICIPANTS)
clean_data['no_previous_contact'] = np.where(clean_data['pdays'] == 999, 1, 0)                                 # Indicator variable to capture when pdays takes a value of 999
clean_data['not_working'] = np.where(np.in1d(clean_data['job'], ['student', 'retired', 'unemployed']), 1, 0)   # Indicator for individuals not actively employed


Don't forget to apply the reformatting steps used for the first tuning job so the data adheres to format requirements of the algorithm.

In [ ]:
# Convert categorical variables to sets of indicators
clean_model_data = pd.get_dummies(clean_data)

# Remove excess target variable column and copy the remaining target variable column to be the first column
clean_model_data = pd.concat([clean_model_data['y_yes'], clean_model_data.drop(['y_no', 'y_yes'], axis=1)], axis=1)

# Split data into 70% training set and 30% validation set
clean_train_data = clean_model_data.sample(frac=0.7, random_state=1729)
clean_validation_data = clean_model_data.drop(clean_train_data.index)

# # Save to CSV format. Drop headers and index.
clean_train_data.to_csv('train.csv', index=False, header=False)
clean_validation_data.to_csv('validation.csv', index=False, header=False)

In [ ]:
# Set the file locations
s3train_clean = 's3://{}/{}/train_2/'.format(bucket, prefix)
s3validation_clean = 's3://{}/{}/validation_2/'.format(bucket, prefix)

# Upload the data to S3
!aws s3 cp train.csv $s3train_clean
!aws s3 cp validation.csv $s3validation_clean

In [ ]:
# Tell SageMaker where to find the input data and where to store the output
s3_output_location = 's3://{}/{}/output_2'.format(bucket, prefix)
s3_input_train = sagemaker.session.s3_input(s3train_clean, distribution='FullyReplicated', 
                        content_type='text/csv', s3_data_type='S3Prefix')
s3_input_validation = sagemaker.session.s3_input(s3validation_clean, distribution='FullyReplicated', 
                             content_type='text/csv', s3_data_type='S3Prefix')

## Launch hyperparameter tuning job using warm start <a class="anchor" id="hyperparam2"></a>

To use warm start in the new tuning job, we need to specify 2 parameters: 
* the list of parent tuning jobs the new tuning job should use as a starting point (The maximum number of parents can be 5).
* the type of warm start configuration:
 * ‘IDENTICAL_DATA_AND_ALGORITHM’ warm starts a tuning job with previous evaluations essentially with the same task, allowing slightly change in the search space. This option should be use when the data set and the algorithm container haven't changed. In this scenario, the only changes to the docker image we recommend are those that do not affect the algorithm, for example changes that only improve logging, or add support of a different data format.
 * ‘TRANSFER_LEARNING’ warm starts a tuning job with the evaluations from similar tasks, allowing both search space, algorithm image and dataset change. 

In this example we will use 'TRANSFER_LEARNING' because we have changed the data.

In [ ]:
from sagemaker.tuner import WarmStartConfig, WarmStartTypes

parent_tuning_job_name = tuning_job_name
warm_start_config = WarmStartConfig(WarmStartTypes.TRANSFER_LEARNING, parents={parent_tuning_job_name})

parent_tuning_job_name

In [ ]:
tuner_warm_start = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            objective_type='Maximize',
                            max_jobs=20,
                            max_parallel_jobs=2,
                            base_tuning_job_name='warmstart',
                            warm_start_config=warm_start_config)

Now we can launch a hyperparameter tuning job by calling tuner.fit and passing warmStartConfig. After the hyperparameter tuning job is created, we can go to SageMaker console to track the progress of the hyperparameter tuning job until it is completed, or run the cell below to print the tuning status.

In [ ]:
tuner_warm_start.fit({'train': s3_input_train, 'validation': s3_input_validation},include_cls_metadata=False)

In [ ]:
# Run this cell to check current status of hyperparameter tuning job
sage_client = boto3.Session().client('sagemaker')
job_name = tuner_warm_start._current_job_name
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
print("Current status: {}".format(status))
if status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
print("{} training jobs have completed".format(job_count))

## Evaluate results of warm start hyperparameter tuning job <a class="anchor" id="eval2"></a>

_You will be unable to successfully run the following cells until the tuning job completes._

Once the tuning job finishes, we can bring in a table of metrics.

In [ ]:
warmstart_tuning_job_name = tuner_warm_start._current_job_name

tuner_warm_start_metrics = sagemaker.HyperparameterTuningJobAnalytics(warmstart_tuning_job_name)
if not tuner_warm_start_metrics.dataframe().empty:
    df_warm_start = tuner_warm_start_metrics.dataframe().sort_values(['FinalObjectiveValue'], ascending=False)

df_warm_start

We then plot the objective metrics for the parent job and the current job.

In [ ]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

import pandas as pd

df_parent_objective_value = df_parent[df_parent['FinalObjectiveValue'] > -float('inf')]
df_warm_start_objective_value = df_warm_start[df_warm_start['FinalObjectiveValue'] > -float('inf')]

p = figure(plot_width=900, plot_height=400, x_axis_type='datetime',x_axis_label='datetime', y_axis_label=objective_metric_name)
p.circle(source=df_parent_objective_value, x='TrainingStartTime', y='FinalObjectiveValue', color='black')
p.circle(source=df_warm_start_objective_value, x='TrainingStartTime', y='FinalObjectiveValue',color='red')
show(p)

Unfortunately, cleaning the data can also result in worse results than the original run. Cleaning is an iterative process and sometimes requires multiple tries to find the right steps.

## Get the best model <a class="anchor" id="best-model"></a>

When the job completes, you can find the training job that generated the best model by using 'BestTrainingJob' in Automatic Model Tuning describe API.

In [ ]:
best_overall_training_job = sage_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=warmstart_tuning_job_name)['BestTrainingJob']

best_overall_training_job

After we have got the best model, we can deploy it to an endpoint. Please refer to other SageMaker sample notebooks or SageMaker documentation to see how to deploy a model.

## Conclusion <a class="anchor" id="conclusion"></a>

In this notebook, we demonstrated how to use warm start to iteratively tune your models.  Warm Start could also be used for other scenarios like: tuning additional hyperparameters, running smaller training jobs (e.g., smaller dataset or fewer epochs) in first tuning job to quickly explore search space then running second tuning job with full size training, or re-tuning a model as you've collected new data over time.

For more information on using SageMaker's Automatic Model Tuning, see our other [example notebooks](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/hyperparameter_tuning) and [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html).